## Importing libraries and modules

In [1]:
# Enable auto-reload
%load_ext autoreload
%autoreload 2

In [2]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Add one level up directory to the path
import sys
sys.path.append("..")

# Import libraries
import torch
import random

# Import custom modules
from src.data import *
from src.machine_translation import *
from src.data.utils import get_dataset
from src.data.preprocess import clean_text
from src.data.tokenizer import CustomBartTokenizer
from src.machine_translation.net import CodeMixedModel, CodeMixedModelHGTrainer
from src.machine_translation.models.bart_conditional import BartForConditionalGeneration
from src.machine_translation.utils import get_tokenized_dataset_models, get_data_loader_models

## Data acquisition, cleaning and processing

In [3]:
%%time
train_df, validation_df, test_df = get_dataset()

CPU times: user 4.58 s, sys: 192 ms, total: 4.77 s
Wall time: 8.04 s


In [4]:
train_df

,hi_en,en
0,film ka kya naam hai,What's the name of the movie
1,"namaste, sada hua tomatoes score mahaan hai, l...","Hi, the rotten tomatoes score is great but the..."
2,kya aapako lagata hai ki aapako film pasand aa...,Do you think you will like the movie
3,yah kis tarah kee philm hai,What kind of movie is it
4,film kab banee thee?,when was the movie made?
...,...,...
8055,Thik hai\n,Ok.
8056,Thik hai bhai\n,ok bro
8057,Kya ham chalu kar sakte hai?\n,shall we continue?
8058,Kya aapko pasand hai hamare saath\n,do you like we can


In [5]:
validation_df

,hi_en,en
0,movie kis baare me hai?,What is the movie about?
1,Movie ek chhote bacche Kevin k baare me hai ji...,the movie is about a young child named Kevin w...
2,Kya wo jaan bhuj k abandon karte hai?,Did they abandon him on purpose?
3,"nahi, wo uska track lose kardete hai kyuki bah...",no they had lost track of him since they had m...
4,Kya wo realize karte hai k wo chhut gaya aur u...,Did they realize they lost track of him and co...
...,...,...
936,Sunkar good movie hai. Kya ham finish kar skat...,Sounds like a good movie. Can we finish now?
937,Yep. Thanks baat karne ke liye\n,Yep. Thanks for chatting
938,"Thanks, mei dekhati hui. Achi baat hai\n","thanks, I will watch it. SOunds good"
940,kya tumhe movie Despicable Me pasand hai?\n,Did you like the movie Despicable Me?


In [6]:
test_df

,hi_en,en
0,oye ledki kitni mazakiya movie dekh rhi ho ?,ah mean girls. such a funny movie. have you se...
1,ha muje bhi hassi mazak ki movies pasand par y...,"Yeah, even though I love comedies, this wasn't..."
2,kese tina ka jhuti writing isme achi the usne ...,how come. thought tina fey's writing was fanta...
3,muje kahani bhute achi lagi aur unhone ise dac...,I loved the story & how true they made it in h...
4,ab samja muje nhi pta tha ki aap sahi admi hai...,I see. i didn't realize it was 14 years ago. y...
...,...,...
6508,alarm ko abhi stop kare,Stop alarm now
6509,Har ghante ke liye alarm set kare,set alarm every hour
6510,Bobby ko text kare,text Bobby
6511,Muje shaam 6 baje laundry ko pick up karne ke ...,remind me to pick up laundry at 6 pm


In [7]:
%%time
train_df = train_df.applymap(clean_text)
train_df

CPU times: user 2.14 s, sys: 2.03 ms, total: 2.14 s
Wall time: 2.14 s


,hi_en,en
0,film ka kya naam hai,what's the name of the movie
1,"namaste, sada hua tomatoes score mahaan hai, l...","hi, the rotten tomatoes score is great but the..."
2,kya aapako lagata hai ki aapako film pasand aa...,do you think you will like the movie
3,yah kis tarah kee philm hai,what kind of movie is it
4,film kab banee thee?,when was the movie made?
...,...,...
8055,thik hai,ok.
8056,thik hai bhai,ok bro
8057,kya ham chalu kar sakte hai?,shall we continue?
8058,kya aapko pasand hai hamare saath,do you like we can


In [8]:
%%time
validation_df = validation_df.applymap(clean_text)
validation_df

CPU times: user 42.4 ms, sys: 1.08 ms, total: 43.5 ms
Wall time: 43 ms


,hi_en,en
0,movie kis baare me hai?,what is the movie about?
1,movie ek chhote bacche kevin k baare me hai ji...,the movie is about a young child named kevin w...
2,kya wo jaan bhuj k abandon karte hai?,did they abandon him on purpose?
3,"nahi, wo uska track lose kardete hai kyuki bah...",no they had lost track of him since they had m...
4,kya wo realize karte hai k wo chhut gaya aur u...,did they realize they lost track of him and co...
...,...,...
936,sunkar good movie hai. kya ham finish kar skat...,sounds like a good movie. can we finish now?
937,yep. thanks baat karne ke liye,yep. thanks for chatting
938,"thanks, mei dekhati hui. achi baat hai","thanks, i will watch it. sounds good"
940,kya tumhe movie despicable me pasand hai?,did you like the movie despicable me?


In [9]:
%%time
test_df = test_df.applymap(clean_text)
test_df

CPU times: user 85.6 ms, sys: 572 µs, total: 86.1 ms
Wall time: 85.7 ms


,hi_en,en
0,oye ledki kitni mazakiya movie dekh rhi ho ?,ah mean girls. such a funny movie. have you se...
1,ha muje bhi hassi mazak ki movies pasand par y...,"yeah, even though i love comedies, this wasn't..."
2,kese tina ka jhuti writing isme achi the usne ...,how come. thought tina fey's writing was fanta...
3,muje kahani bhute achi lagi aur unhone ise dac...,i loved the story & how true they made it in h...
4,ab samja muje nhi pta tha ki aap sahi admi hai...,i see. i didn't realize it was 14 years ago. y...
...,...,...
6508,alarm ko abhi stop kare,stop alarm now
6509,har ghante ke liye alarm set kare,set alarm every hour
6510,bobby ko text kare,text bobby
6511,muje shaam 6 baje laundry ko pick up karne ke ...,remind me to pick up laundry at 6 pm


## Data tokenization

In [10]:
# Now that we know how to get a BART-Tokenizer - default, appeded and scratch, let us get the tokenizer for our code-mixed language
# and the target language.
# Code-Mixed Language - Build a Bart-Tokenizer from scratch.
# Target Language (English) - Use the default Bart-Tokenizer.
hi_en_bart_tokenizer = CustomBartTokenizer().build(
    data=train_df["hi_en"],
    tokenizer_style=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_ENCODER_STYLE,
    tokenizer_bart_from_pretrained_path=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_ENCODER_FROM_PRETRAINED
)
en_bart_tokenizer = CustomBartTokenizer().build(
    tokenizer_style=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_DECODER_STYLE,
    tokenizer_bart_from_pretrained_path=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_DECODER_FROM_PRETRAINED
)

In [11]:
print(f"Hinglish tokenizer vocab size: {hi_en_bart_tokenizer.vocab_size}")
print(f"English tokenizer vocab size: {en_bart_tokenizer.vocab_size}")

Hinglish tokenizer vocab size: 50265
English tokenizer vocab size: 50265


## Dataset and Data Loaders

## Model definition

## Model Training

In [12]:
dataset = get_tokenized_dataset_models(
    train_df=train_df,
    validation_df=validation_df,
    test_df=test_df,
    encoder_tokenizer=hi_en_bart_tokenizer,
    decoder_tokenizer=en_bart_tokenizer
)
train_dataset, validation_dataset, test_dataset = dataset["hi_en__en"].values()

In [17]:
mt_hg_model = CodeMixedModelHGTrainer(
    train_dataset = train_dataset,
    validation_dataset = validation_dataset,
    test_dataset = test_dataset,
    encoder_tokenizer=hi_en_bart_tokenizer,
    decoder_tokenizer=en_bart_tokenizer
)

Getting the model...

Configuring optimizer and scheduler...

Configuring collator...

Configuring trainer...

Configuring training arguments...

Configuring generate config...


In [18]:
mt_hg_model.fit()


Training the model...


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Step,Training Loss,Validation Loss


***** train metrics *****
  epoch                    =       0.09
  total_flos               =   789964GF
  train_loss               =     0.0004
  train_runtime            = 0:00:01.12
  train_samples            =     174443
  train_samples_per_second =   7762.959
  train_steps_per_second   =    242.978

Validating the model...


Evaluating bleu score: 100%|█| 102/102 [00

***** validation metrics *****
  bleu               = 0.8301
  validation_samples =   3243


In [19]:
mt_hg_model.predict()


Generating translations...



Predicting strings: 100%|█| 221/221 [00:40

BLEU Score on the test data:  0.9181546059233954
